# 自然灾害遥感——基于MODIS数据的河南旱情监测与分析
- 本课题以河南省为研究范围，以TVDI指数为核心，对其干湿边拟合方式及不同的温度数据来源进行了讨论，选择最优干旱监测模型对2014年河南省旱情进行了监测
- 基于Google Earth Engine平台与本地Python的numpy，geemap等数据包对经过云平台处理的遥感影像进行处理分析
- 数据源使用MODIS的NDVI,EVI及温度产品，时空分辨率约为1000m

## 0.实验准备

In [ ]:
#导入Geemap库
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
#导入研究区
henan=ee.FeatureCollection('users/gjsinfluence/HeNan')
Map.addLayer(henan, {}, "henanROI")
Map.setCenter(116.186, 34.591, 6)

In [ ]:
#可视化参数设置
ndviVis = {
    'min': 0.0,
    'max': 9000.0,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301'
    ]
  }
LAIVis = {
    'min': 0.0,
    'max': 100.0,
    #palette: ['e1e4b4', '999d60', '2ec409', '0a4b06'],
      'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301'
    ]
  }
LSTVis = {
    'min': 14000.0,
    'max': 16000.0,
    'palette': [
      '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
      '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
      '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
      'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
      'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
    }
dem_vis = {
'min': 0,
'max': 4000,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

In [ ]:
#设置起始时间及间隔
startDate=ee.Date('2014-6-01')
frequency=9#每半月为一个间隔，用于数据获取

## 1.云平台数据获取

### 1.1NDVI与EVI的数据获取

In [ ]:
#NDVI与EVI的数据获取
NDVIList=[]
EVIList=[]
temStartDate=startDate
for i in range(frequency):
    temEndDate=temStartDate.advance(0.5,'month')
    print('StartDate:',temStartDate.format().getInfo(),'Enddate:',temEndDate.format().getInfo())
    dataset=ee.ImageCollection('MODIS/061/MOD13A1')\
                    .filter(ee.Filter.date(temStartDate, temEndDate))\
                    .filterBounds(henan)\
                    .max()\
                    .clip(henan)
    NDVI=dataset.select('NDVI')
    NDVIList.append(NDVI)
    EVI=dataset.select('EVI')
    EVIList.append(EVI)
    temStartDate=temEndDate
print('NDVInum:',len(NDVIList))
print('EVInum:',len(EVIList))

### 1.2LAI的数据获取

In [ ]:
#LAI的数据获取
LAIList=[]
temStartDate=startDate
for i in range(frequency):
    temEndDate=temStartDate.advance(0.5,'month')
    print('StartDate:',temStartDate.format().getInfo(),'Enddate:',temEndDate.format().getInfo())
    dataset=ee.ImageCollection('MODIS/061/MCD15A3H')\
                    .filter(ee.Filter.date(temStartDate, temEndDate))\
                    .filterBounds(henan)\
                    .mosaic()\
                    .clip(henan)
    LAI=dataset.select('Lai')
    LAIList.append(LAI)
    temStartDate=temEndDate
print('LAInum:',len(LAIList))

### 1.3 地表温度的数据获取

In [ ]:
#质量控制函数，利用MODIS数据产品的QA波段进行质量控制
def bitwiseExtract(input,fromBit,toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask=ee.Number(1).leftShift(maskSize).subtract(1)
    return input.rightShift(fromBit).bitwiseAnd(mask)
#对日温进行控制，输入日温影像
def qcControlLSTDay(image):
    qcDay=image.select('QC_Day')
    lstDay=image.select('LST_Day_1km')
    qaMask=bitwiseExtract(qcDay,0,1).lte(1)
    dataQualityMask=bitwiseExtract(qcDay,2,3).eq(0)
    lstErrorMask=bitwiseExtract(qcDay,6,7).eq(0)
    mask=qaMask.And(dataQualityMask).And(lstErrorMask)
    return lstDay.updateMask(mask)
#对夜温进行质量控制，输入夜温影像
def qcControlLSTNight(image):
    qcDay=image.select('QC_Night')
    lstDay=image.select('LST_Night_1km')
    qaMask=bitwiseExtract(qcDay,0,1).lte(1)
    dataQualityMask=bitwiseExtract(qcDay,2,3).eq(0)
    lstErrorMask=bitwiseExtract(qcDay,6,7).eq(0)
    mask=qaMask.And(dataQualityMask).And(lstErrorMask)
    return lstDay.updateMask(mask)

In [ ]:
#获取 地表温度的日温、昼夜温差、日夜平均气温数据
LST_DAY_List=[]
LST_Night_List=[]
LST_Mean_List=[]
LST_Differ_List=[]
temStartDate=ee.Date('2014-6-01')
DEMdataset = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(henan)
DEM = DEMdataset.select('DEM').mosaic().clip(henan)
for i in range(frequency):
    temEndDate=temStartDate.advance(0.5,'month')
    print('StartDate:',temStartDate.format().getInfo(),'Enddate:',temEndDate.format().getInfo())
    LSTdataset=ee.ImageCollection('MODIS/006/MOD11A1')\
                    .filter(ee.Filter.date(temStartDate, temEndDate))\
                    .filterBounds(henan)
    LST_DAY=LSTdataset.map(qcControlLSTDay)\
                        .max()\
                        .clip(henan)\
                        .subtract(DEM.multiply(0.3))#日温
    LST_Night=LSTdataset.map(qcControlLSTNight).max().clip(henan).subtract(DEM.multiply(0.3))
    LST_Mean=LST_DAY.add(LST_Night).divide(2)#日夜平均气温
    LST_Differ=LST_DAY.subtract(LST_Night)#日夜温差
    LST_DAY_List.append(LST_DAY)
    LST_Night_List.append(LST_Night)
    LST_Mean_List.append(LST_Mean)
    LST_Differ_List.append(LST_Differ)
    temStartDate=temEndDate
print('LST_DAY_List:',len(LST_DAY_List))
print('LST_Night_List:',len(LST_Night_List))
print('LST_Mean_List:',len(LST_Mean_List))
print('LST_Differ_List:',len(LST_Differ_List))

### 1.4 数据导出到本地

In [ ]:
#使用geemap的导出函数进行，可以导出至GoogleDrive，也可直接下载到本地，
#可参照https://geemap.org/cheatsheet/#export-data
#下述代码使用了日期作为图像名称进行了格式化输出，输出至GoogleDrive
temStartDate=ee.Date('2014-6-01')
temEndDate=ee.Date('2014-6-01')
for i in range(0,9):
    print(i)
    temEndDate=temStartDate.advance(0.5,'month')
    Date_str=temStartDate.format().getInfo()[5:10]+'to'+temEndDate.format().getInfo()[5:10]
    print(Date_str)
    LST_DAY_name="LST_DAY_%s"%(Date_str)
    print(LST_DAY_name)
    geemap.ee_export_image_to_drive(LST_DAY_List[i], description=LST_DAY_name, folder='export', region=henan.geometry(), scale=1000)
    LST_Night_name="LST_Night_%s"%(Date_str)
    print(LST_Night_name)
    geemap.ee_export_image_to_drive(LST_Night_List[i], description=LST_Night_name, folder='export', region=henan.geometry(), scale=1000)
    LST_Mean_name="LST_Mean_%s"%(Date_str)
    print(LST_Mean_name)
    geemap.ee_export_image_to_drive(LST_Mean_List[i], description=LST_Mean_name, folder='export', region=henan.geometry(), scale=1000)
    LST_Differ_name="LST_Differ_%s"%(Date_str)
    print(LST_Differ_name)
    geemap.ee_export_image_to_drive(LST_Differ_List[i], description=LST_Differ_name, folder='export', region=henan.geometry(), scale=1000)
    NDVIname="NDVI_%s"%(Date_str)
    print(NDVIname)
    geemap.ee_export_image_to_drive(NDVIList[i], description=NDVIname, folder='export', region=henan.geometry(), scale=1000)
    EVIname="EVI_%s"%(Date_str)
    print(EVIname)
    geemap.ee_export_image_to_drive(EVIList[i], description=EVIname, folder='export', region=henan.geometry(), scale=1000)
    temStartDate=temEndDate

# 2.进行反演

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import leastsq
import os
from osgeo import gdal
from osgeo import osr
import matplotlib.ticker as mticker

### 2.1读写文件的相关函数定义

In [ ]:
#读写文件
#读取tif数据集
def readTif(fileName):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName+"文件无法打开")
    # else print(fileName+"文件打开成功")
    return dataset

#保存tif文件函数
'''
description: 描述
param {*} im_data       源数组数据
param {*} im_geotrans   地理信息
param {*} im_proj       投影信息
param {*} path          保存路径
return {*}
'''
def writeTiff(im_data,im_geotrans,im_proj,path):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
        im_bands, im_height, im_width = im_data.shape
    #创建文件
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, int(im_width), int(im_height), int(im_bands), datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans) #写入仿射变换参数
        dataset.SetProjection(im_proj) #写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i+1).WriteArray(im_data[i])
    del dataset

In [ ]:
#判断是否存在文件夹如果不存在则创建为文件夹
def mkdir(path):
	print(path)
	folder = os.path.exists(path)
	if not folder:                   
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径

### 2.2 干湿边线性拟合的函数定义

In [ ]:
#进行干湿边线性拟合并绘图
'''
description: 描述   绘制干湿边图像
param {*} NDVI      输入NDVI影像
param {*} emax      输入的温度最大值
param {*} emin      输入的温度最小值
param {*} order     进行拟合的次数，可为1,2阶
param {*} outfig    输出的影像路径
param {*} Datestr   影像的日期字符串，便于命名
param {*} VIname    植被指数的名称。如EVI,NDVI
param {*} LSTname   温度的名称，如日温LST_Day，均温LST_Mean
return {*}          返回干湿边拟合参数及R2
'''
def createPlot(NDVI, emax, emin,order,outfig,Datestr,VIname,LSTname):
    plt.rcParams['font.family'] = 'Times New Roman'
    fig, ax = plt.subplots(figsize=(9, 6))
    plt.tick_params(labelsize=16)

    #  设置y轴、x轴上下限
    ylimit_min = np.min(emin) - 0.1 * np.average(emin)
    ylimit_max = np.max(emax) + 0.1 * np.average(emax)
    plt.ylim(ylimit_min, ylimit_max)
    plt.xlim(0, 1)
    
    #  画出干边湿边的拟合方程
    cof = np.polyfit(NDVI, emax, order)#进行拟合的干湿边方程参数表，1为阶数
    dryp = np.poly1d(cof)
    #进行平滑处理
    step = 0.01
    _x = np.arange(np.min(NDVI), np.max(NDVI), step)
    _y=dryp(_x)
    #计算r2
    dryR2 = np.corrcoef(emax, dryp(NDVI))[0,1]  #相关系数
    dryR2**2   #R方
    # plt.plot(NDVI, p(NDVI), label="Dry edge fitted line \n test", c='g')
    # plt.plot(_x, _y, label="Dry edge fitted line \n $%dx^2+%dx+%d$" %(cof[0],cof[1],cof[2]), c='g')#加入\n可以实现换行，以及插入公式
    plt.plot(_x, _y, label="Dry edge fitted line ", c='g')#加入\n可以实现换行
    plt.plot(label='test')

    cof = np.polyfit(NDVI, emin, order)
    wetp = np.poly1d(cof)
    step = 0.01
    _x = np.arange(np.min(NDVI), np.max(NDVI), step)
    _y=wetp(_x)  
    #计算r2
    wetR2 = np.corrcoef(emin, wetp(NDVI))[0,1]  #相关系数
    wetR2**2   #R方  
    plt.plot(_x, _y, label='Wet edge fitted line', c='y')
    #  画出干边湿边的散点
    plt.scatter(NDVI, emax, label='Dry edge', c = 'r')
    plt.scatter(NDVI, emin, label='Wet edge', c = 'b')
    #  设置y轴、x轴标注
    ax.set_xlabel(VIname, fontsize=16)
    # ax.set_ylabel('LST ($^o$C)', fontsize=16)
    ax.set_ylabel(LSTname+" "+"(K)", fontsize=16)
    plt.legend(fontsize=16, loc='upper right', ncol=2)
    fig.tight_layout()
    plt.title("VI%s-%sLST%sSpace（%s）"%(' ',' ',' ', Datestr), fontproperties="simhei",fontsize=18)
    plt.savefig(outfig, dpi=300,bbox_inches='tight')
    plt.subplots_adjust()
    plt.show()
    return dryp,wetp,dryR2,wetR2


### 2.3去除读取的Nan无效像元值的函数定义

In [ ]:
#去除nan无效值
'''
description: 描述   以data1的nan数据位置为参照，去除data1，data2，data3相应数据
param {*} data1     参照nan数据  
param {*} data2     待去除数据1
param {*} data4     待去除数据2
return {*}          去除后的三组数据
'''
def removeNan(data1,data2,data4):
    data3=np.array([])
    num=len(data1)
    for i in range(num) :
        if (math.isnan(data1[i])):
            data3=np.append(data3,i)
    data3=data3[::-1]
    num1=len(data3)
    for i in range(num1):
        data1=np.delete(data1,int(data3[i]))
        data2=np.delete(data2,int(data3[i]))
        data4=np.delete(data4,int(data3[i]))
        # data5=np.delete(data5,int(data3[i]))
    return data1,data2,data4
#去除单个的数组nan
def removeNan1(data1):
    data3=np.array([])
    num=len(data1)
    for i in range(num) :
        if (math.isnan(data1[i])):
            data3=np.append(data3,i)
    data3=data3[::-1]
    num1=len(data3)
    for i in range(num1):
        data1=np.delete(data1,int(data3[i]))
    return data1

### 2.4记性干湿边拟合的函数定义

In [ ]:
#进行干湿边拟合
'''
description: 描述       进行干湿边拟合
param {*} NDVI          输入的植被指数信息
param {*} LST           输入的温度信息
param {*} NDVI_range    输入的NDVI拟合范围
param {*} outfig        输出的文件保存路径
param {*} order         拟合次数
param {*} Datestr       日期字符串，用以命名
param {*} VIname        植被指数类型，如NDVI,EVI
param {*} LSTname       温度类型，如LST_Differ等
return {*}
'''
def edge(NDVI, LST, NDVI_range, outfig,order,Datestr,VIname,LSTname):
    print("干边、湿边拟合...")
    f = open('D:/ThermalRS/Google/out.txt','w')
    edgeArr = [[] for _ in range(99)]
    for i in range(NDVI.shape[0]):
        for j in range(NDVI.shape[1]):
            if (NDVI[i][j] > NDVI_range[0] - 0.01 and NDVI[i][j] < NDVI_range[1] + 0.01 ):
                if(math.isnan(NDVI[i][j])):
                    continue
                k = NDVI[i][j]
                print(int(float(k) * 100),file=f)
                edgeArr[int(float(k) * 100)].append(LST[i][j])
    f.close()      
    # 排序，寻找干边湿边
    # print("排序，寻找干边湿边")
    eMax = []
    NDVISort = []
    eMin = []
    for n in range(len(edgeArr)):
        edgedata=removeNan1(edgeArr[n])
        if len(edgedata) > 1:
            max=np.max(edgedata)
            eMax.append(max)
            min=np.min(edgedata)
            eMin.append(min)
            NDVISort.append(float(n) / 100.)
    eMax,NDVISort,eMin=removeNan(eMax,NDVISort,eMin)
    dryp,wetp,dryR2,wetR2=createPlot(NDVISort, eMax, eMin,order, outfig,Datestr,VIname,LSTname)
    return dryp,wetp,dryR2,wetR2

### 2.5进行栅格逐像元计算

In [ ]:

#进行计算
import xlsxwriter
#formulaSettings:[xlsx路径文件,组合的字符串,起始行数,VI指数名，LST类型]
def calTVDI(NDVIPath, LSTPath, NDVI_range, order,outfig, SavePath,Datestr,formulaSettings,progressFile):
    #输出相关配置文件
    print('NDVIPath:'+NDVIPath,file=progressFile)
    print("LSTPath:"+LSTPath,file=progressFile)
    print("outfig:"+outfig,file=progressFile)
    print('SavePath:'+SavePath,file=progressFile)
    # 读取输入栅格数据信息
    print("读取输入栅格数据信息...")
    dataset = readTif(NDVIPath)
    width = dataset.RasterXSize #栅格矩阵的列数
    height = dataset.RasterYSize #栅格矩阵的行数
    NDVI_geotrans = dataset.GetGeoTransform()
    NDVI_proj = dataset.GetProjection()
    NDVI = dataset.ReadAsArray(0,0,width,height)#获取数据
    NDVI=NDVI/10000
    dataset = readTif(LSTPath)
    LST = dataset.ReadAsArray(0,0,width,height)#获取数据
    LST=LST*0.02
    # LST = LST - 273
    dryp,wetp,dryR2,wetR2= edge(NDVI, LST, NDVI_range, outfig,order,Datestr,formulaSettings[3],formulaSettings[4])
    #保存方程文件
    worksheet=formulaSettings[0]
    n=formulaSettings[2]
    if(order==1):
        print("干边拟合方程：y = %.3fx + %.3f，r2 = %.3f" % (dryp[0], dryp[1], dryR2))
        print("湿边拟合方程：y = %.3fx + %.3f，r2 = %.3f" % (wetp[0],wetp[1], wetR2))
        worksheet.write(n,0,Datestr)
        worksheet.write(n,1,'干边拟合方程：')
        worksheet.write(n,2,'y=%.3fx+%.3f'%(dryp[0], dryp[1]))
        worksheet.write(n,3,'R^2:')
        worksheet.write(n,4,dryR2)
        n=n+1
        worksheet.write(n,1,'湿边拟合方程：')
        worksheet.write(n,2,'y=%.3fx+%.3f'%(wetp[0], wetp[1]))
        worksheet.write(n,3,'R^2:')
        worksheet.write(n,4,wetR2)
        # print("%s;干边拟合方程：;y=%.3fx+%.3f;R^2:;%.3f"% (Datestr,dryp[0], dryp[1], dryR2),file=f)
        # print("%s;湿边拟合方程：;y=%.3fx+%.3f;R^2:;%.3f"% (' ',wetp[0], wetp[1], wetR2),file=f)
        
    if(order==2):
        print("干边拟合方程：y=%.3fx^2+%.3fx+%.3f，r2 = %.3f" % (dryp[0], dryp[1],dryp[2], dryR2))
        print("湿边拟合方程：y = %.3fx^2 + %.3fx + %.3f，r2 = %.3f" % (wetp[0],wetp[1],wetp[2], wetR2))
        worksheet.write(n,0,Datestr)
        worksheet.write(n,1,'干边拟合方程：')
        worksheet.write(n,2,'y=%.3fx^2+%.3fx+%.3f'%(dryp[0], dryp[1],dryp[2]))
        worksheet.write(n,3,'R^2:')
        worksheet.write(n,4,dryR2)
        n=n+1
        worksheet.write(n,1,'湿边拟合方程：')
        worksheet.write(n,2,'y=%.3fx^2+%.3fx+%.3f'%(wetp[0], wetp[1],wetp[2]))
        worksheet.write(n,3,'R^2:')
        worksheet.write(n,4,wetR2)
        # print("%s;干边拟合方程：;y=%.3fx^2+%.3fx+%.3f;R^2:;%.3f"% (Datestr,dryp[0], dryp[1],dryp[2], dryR2),file=f)
        # print("%s;湿边拟合方程：;y=%.3fx^2+%.3fx+%.3f;R^2:;%.3f"% (' ',wetp[0], wetp[1],wetp[2], wetR2),file=f)
    print("计算TVDI...")
    TVDI = np.zeros(NDVI.shape)
    for m in range(NDVI.shape[0]):
        for n in range(NDVI.shape[1]):
            TVDI[m][n] = (LST[m][n] - dryp(NDVI[m][n])) / (wetp(NDVI[m][n]) - dryp(NDVI[m][n]))
    TVDI[TVDI < 0] = 0
    TVDI[TVDI > 1] = 1
    #  保存TVDI
    writeTiff(TVDI,NDVI_geotrans,NDVI_proj,SavePath)
    print("计算完成,保存为 '%s'\n" % SavePath)

### 2.6 进行函数反演

In [ ]:
#文件路径设置
VIFolder = r"D:/ThermalRS/NEWIMG/RAWIMG/VI/"#植被指数影像的文件夹
LSTFolder = r"D:/ThermalRS/NEWIMG/RAWIMG/LST/"#温度的影像文件夹
SavePath=r"D:/ThermalRS/NEWIMG/OUT/"#反演结果矢量的保存路径
progressfile=open('D:/ThermalRS/NEWIMG/progressfile.txt','w')#记录输入与输出的日志，便于后续查阅
workbook = xlsxwriter.Workbook('D:/ThermalRS/NEWIMG/OUT/testformula.xlsx')#打开excel，输出所拟合的方程信息


In [ ]:
NDVI_range = [0.1, 0.9]#设置进行干湿边拟合的NDVI范围
for i in range(len(VINameList)):
    for j in range(len(LSTNameList)):
        combinestr=VINameList[i]+'_'+LSTNameList[j]#组合名称
        print("ALLcombine:",combinestr)
        combinePath=SavePath+combinestr#创建不同类别组合的文件夹
        mkdir(combinePath)
        combineOrderPath1=combinePath+'/'+"一次拟合结果"#创建不同类别组合不同次数的文件夹
        combineOrderPath2=combinePath+'/'+"二次拟合结果"
        mkdir(combineOrderPath1)
        mkdir(combineOrderPath2)
        #创建不同的拟合方式路径数组
        orderpath=[combineOrderPath1,combineOrderPath2]
        VIPath = VIFolder  + VINameList[i]
        LSTPath=LSTFolder+LSTNameList[j]
        VIFileList = os.listdir(VIPath)#[EVI_0601-0616.tif,]
        LSFileList = os.listdir(LSTPath)#[LST_0601-0616.tif]
        for n in range(2):#设置拟合次数
            n1=n+1
            combinestr1=combinestr+str(n1)
            worksheet = workbook.add_worksheet(combinestr1)
            for x in range(len(VIFileList)):
                # print("combine:",combinestr1)
                #读取时间信息
                beginDateStr=str(VIFileList[x]).split('.')[0].split('to')[0].split('_')[1]
                endDateStr=str(VIFileList[x]).split('.')[0].split('to')[1]
                Datestr=beginDateStr+'至'+endDateStr
                # print(Datestr)
                #读取正式文件路径
                VIFile=VIPath+'/'+VIFileList[x]
                LSTFile=LSTPath+"/"+LSFileList[x]
                # print(VIFile)
                # print(LSTFile)
                # print("xlsx:",combinestr1)
                SavePath1=orderpath[n]+'/'+"RAWTIF/"
                mkdir(SavePath1)
                SavePath2=SavePath1+combinestr1+'_'+Datestr+'.tif'
                # print('SavePath:',SavePath2)
                outfig=orderpath[n]+'/'+"PNG/"
                mkdir(outfig)
                outfig1=outfig+combinestr1+'_'+Datestr+'.png'
                # print('outfig:',outfig1)
                formulaSettings=[worksheet,combinestr1,2*x,VINameList[i],LSTNameList[i]]
                # print(2*x)
                calTVDI(VIFile, LSTFile, NDVI_range, n1,outfig1, SavePath2,Datestr,formulaSettings,progressfile)
            render(SavePath1)
progressfile.close()      
workbook.close()#写入信息


# 3.精度评估
- 有实测数据可以进行插值拟合河南省影像进行对比验证
- 这里无实测数据，选择了Google Earth Engine的相同时间段的降水产品GPM进行对比验证

### 3.1获取GPM数据

In [ ]:
GPMList=[]
temStartDate=ee.Date('2014-6-01')
for i in range(9):
    temEndDate=temStartDate.advance(0.5,'month')
    print('StartDate:',temStartDate.format().getInfo(),'Enddate:',temEndDate.format().getInfo())
    dataset=ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
                    .filter(ee.Filter.date(temStartDate, temEndDate))\
                    .filterBounds(henan)\
                    .mean()\
                    .clip(henan)
#     print(dataset.bandNames().getInfo())
    GPM=dataset.select('precipitation')
    GPMList.append(GPM)
    temStartDate=temEndDate
print('GPMNUM:',len(GPMList))

### 3.2进行数据导出

In [ ]:
temStartDate=ee.Date('2014-6-01')
temEndDate=ee.Date('2014-6-01')
for i in range(0,9):
    print(i)
    temEndDate=temStartDate.advance(0.5,'month')
    Date_str=temStartDate.format().getInfo()[5:10]+'to'+temEndDate.format().getInfo()[5:10]
    print(Date_str)
    CPM_name="CPM_%s"%(Date_str)
    print(CPM_name)
    geemap.ee_export_image_to_drive(GPMList[i], description=CPM_name, folder='export', region=henan.geometry(), scale=5566)
#     Map.addLayer(GPMList[i], CPMVis, Date_str);
    temStartDate=temEndDate

# 4.制图与渲染
- 使用cartopy库对生成的tiff数据进行了渲染

In [ ]:
def render(TVDIFolder):
    TVDINameList = os.listdir(TVDIFolder)
    outPath=TVDIFolder+'Render'
    mkdir(outPath)
    for i in range(len(TVDINameList)):
        # name=TVDINameList[i].split('.')[0]
        name="Precipitation_Mean_"+TVDINameList[i].split('.')[0].split('_')[1]
        TVDIPath = TVDIFolder + "/" + TVDINameList[i]
        dataset = readTif(TVDIPath)
        width = dataset.RasterXSize #栅格矩阵的列数
        height = dataset.RasterYSize #栅格矩阵的行数
        TVDI = dataset.ReadAsArray(0,0,width,height)#获取数据
        plt.clf()
        sc = plt.imshow(TVDI, cmap = plt.cm.jet)# 设置cmap为RGB图
        plt.colorbar()# 显示色度条
        # plt.title(str(name),fontproperties="simhei")
        plt.title(str(name))
        plt.savefig(outPath + "/" + name+ "_render.png", dpi=300)
        plt.show()